# Darcy Flow
Including forward and inverse problem solver for 1D. Darcy Flow
Darcy Flow
\begin{equation}
    \nabla \cdot ( k(x) \nabla u(x) ) = f(x)
\end{equation}

Zero Dirichlet boundary conditions $u(0) = u(1) = 0$

## Hints from Tim Sullivan

Express $u$ as $u(x) = \sum_{j = 1}^{n} u_{j} \phi_{j}(x)$ with $\phi_{j}(x) =$ piecewise linear tent function peaking at node $j$

Solve $A(u_{1}, …, u_{n}) = b$ for the coefficients  
\begin{align}
b_{j} &= \int_{0}^{1} f(x) \phi_{j}(x) d x \\
A_{i j} &= \int_{0}^{1} k(x) \phi'_{i}(x) \phi'_{j}(x) d x
\end{align}
Google finite element method / Galerkin method for elliptic PDE 

I will give you e.g. (u(1/4), u(1/2), u(3/4)) and similarly for f, both corrupted by additive N(0, \sigma^{2}) noise.
Your challenge:  infer k

--

Modelling assumption:  $k(x) = exp( \sum_{\alpha = 0}^{A} k_{\alpha} \phi_{\alpha}(x) )$

--

Try a Fourier basis. Note that u vanishes at the boundary, but that doesn't mean k does.

Create the forward model. Solver works. Extend to imitation inverse model with likelihood, etc. Use this to solve his problem.


# The Darcy Flow Problem

We consider the Darcy Flow problem in one dimension with Dirichlet boundry conditions and a modeling assumption.

\begin{align}
    k'(x) u'(x) + k(x) u''(x) &= f(x) \\
    u(0) = u(1) &= 0 \\
    exp( \sum_{\alpha = 0}^{A} k_{\alpha} \phi_{\alpha}(x) ) &=  k(x)
\end{align}

We put the equation into the weak form. $v(x)$ is a function which satisfies the boundary conditions.

\begin{equation}
    \int_{0}^{1} f(x)v(x) \ dx = \int_{0}^{1} (k'(x) u'(x) v(x) + k(x) u''(x) v(x)) \ dx
\end{equation}

We can integrate by parts and apply our boundry conditions on the second R.H.S. term to arrive at the next equation.

\begin{align}
    \int_{0}^{1} f(x)v(x) \ dx &= \int_{0}^{1} k'(x) u'(x) v(x) \ dx +
                                  k(x) u'(x) v(x) |_{0}^{1} - 
                                  \int_{0}^{1} k'(x) u'(x) v(x) \ dx - 
                                  \int_{0}^{1} k(x) u'(x) v'(x) \ dx \\
    \int_{0}^{1} f(x)v(x) \ dx &= - \int_{0}^{1} k(x) u'(x) v'(x) \ dx
\end{align}

We choose the piecewise linear function $v_k$ for a discretization.

\begin{equation}
v_{k}(x) = 
    \begin{cases}
        \frac{x-x_{k-1}}{x_k-x_{k-1}} & \text{if } x \in [x_{k-1}, x_k] \\
        \frac{x_{k+1}-x}{x_{k+1}-x_k} & \text{if } x \in [x_{k}, x_{k+1}] \\
        0 & \text{otherwise}
    \end{cases}
\end{equation}

If we expand $u(x)$ in a basis of tent functions on this discretization, we are left with the problem

\begin{equation}
    A \bf{u} = \bf{b}
\end{equation}

where $A_{ij} = - \int_{0}^{1} k(x) v_{i}'(x) v_{j}'(x) \ dx$ and $b_{j} = \int_{0}^{1} f(x) v_{j} dx$. Note that this will be a sparse matrix due our use of tent functions. With our modeling assumption, only 

## Useful Links
https://en.wikipedia.org/wiki/Finite_element_method  
http://www.mathematik.uni-dortmund.de/~kuzmin/Transport.pdf  

# Forward Solver

In [ ]:
import numpy as np
import scipy.integrate
import matplotlib.pyplot as plt

In [ ]:
def grid(nodes):
    return np.linspace(0, 1, num=nodes)
    
def tent(x, k, grid):
    def down(x):
        return (grid[k+1] - x) / (grid[k+1] - grid[k])
    def up(x):
        return (x - grid[k-1]) / (grid[k] - grid[k-1])
    if (k < 0) or (k > grid.size - 1):
        raise ValueError('k was not in [0, grid.size - 1]')
    elif k == 0:
        if (grid[k] <= x) and (x <= grid[k+1]):
            return down(x)
        else:
            return 0
    elif k == grid.size - 1:
        if (grid[k-1] <= x) and (x <= grid[k]):
            return up(x)
        else:
            return 0
    else:
        if (grid[k-1] <= x) and (x <= grid[k]):
            return up(x)
        elif (grid[k] <= x) and (x <= grid[k+1]):
            return down(x)
        else:
            return 0
        
def κ(x, coefs, grid):
    lo_bound = np.searchsorted(grid, x, 'left')
    up_bound = np.searchsorted(grid, x, 'right')
    
    if (0 - 0.1 <= x) and (x < grid[1]):
        return np.exp(sum([coefs[0] * tent(x, 0, grid), coefs[1] * tent(x, 1, grid)]))
    elif (grid[-2] < x) and (x <= grid[-1] + 0.1):
        return np.exp(sum([coefs[-2] * tent(x, grid.size - 2, grid), coefs[-1] * tent(x, grid.size - 1, grid)]))
    else:
        return np.exp(sum([coefs[k] * tent(x, k, grid) for k in [lo_bound, up_bound]]))
    
# def kappa(x, coefs, grid):
#     return np.exp(sum([coefs[k] * tent(x, k, grid) for k in range(grid.size)]))

def A(k, grid):
    val = lambda x, y: sp.integrate.quad(k, x, y, limit=100)[0]
    lo_di = np.asarray([ val(grid[i-1], grid[i]  ) for i in range(2, grid.size - 1)])
    di    = np.asarray([-val(grid[i-1], grid[i+1]) for i in range(1, grid.size - 1)])
    up_di = np.asarray([ val(grid[i]  , grid[i+1]) for i in range(1, grid.size - 2)])
    
    return np.sum([np.diag(lo_di, -1), np.diag(di), np.diag(up_di, 1)], axis=0)

def b(f, grid):
    val = lambda x, y: sp.integrate.quad(f, x, y)[0]
    return np.asarray([-val(grid[i-1], grid[i+1]) for i in range(1, grid.size - 1)])

In [ ]:
nodes = 5
g = grid(nodes)
plt.plot([κ(x, np.random.rand(nodes), g) for x in np.linspace(0, 1, num=100)])

for i in range(nodes):
    plt.plot([tent(x, i, g) for x in np.linspace(0, 1, num=100)])
plt.show()

In [ ]:
nodes = 10
pars = np.random.randn(nodes)
g = grid(nodes)
aa = A(lambda x: κ(x, pars, g), g)
bb = b(lambda x: np.cos(x), g)

np.linalg.solve(aa, bb)